# Task 2 – Time Series Forecasting Models

**Asset:** TSLA Close Price  
**Train:** 2015-01-01 → 2024-12-31  
**Test:** 2025-01-01 → 2026-01-15  

### Sections
1. Data Preparation & Train/Test Split  
2. ARIMA / SARIMA  
3. LSTM  
4. Evaluation – MAE, RMSE, MAPE  
5. Model Comparison & Discussion  
6. Save Outputs

## 1. Data Preparation & Train/Test Split

In [ ]:
# ── Imports ───────────────────────────────────────────────────────────────────
from __future__ import annotations

import json
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA as SmARIMA
from statsmodels.tsa.stattools import adfuller
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

warnings.filterwarnings("ignore")
tf.random.set_seed(42)
np.random.seed(42)

# ── Paths ─────────────────────────────────────────────────────────────────────
NOTEBOOK_DIR = Path(".")
DATA_DIR     = NOTEBOOK_DIR.parent / "data" / "processed"
IMG_DIR      = NOTEBOOK_DIR / "images"
IMG_DIR.mkdir(parents=True, exist_ok=True)

TRAIN_END  = "2024-12-31"
TEST_START = "2025-01-01"

print("TensorFlow:", tf.__version__)
print("DATA_DIR exists:", DATA_DIR.exists())
print("IMG_DIR:", IMG_DIR.resolve())


In [ ]:
# ── Load TSLA CSV & split ─────────────────────────────────────────────────────
tsla_raw = pd.read_csv(DATA_DIR / "TSLA_clean.csv", header=[0, 1], index_col=0)
tsla_raw.columns = tsla_raw.columns.get_level_values(0)
tsla_raw = tsla_raw.dropna(how="all")
tsla_raw.index = pd.to_datetime(tsla_raw.index, errors="coerce")
tsla_raw = tsla_raw[tsla_raw.index.notna()].sort_index()

close: pd.Series = tsla_raw["Close"].astype(float).dropna()

train: pd.Series = close.loc[:TRAIN_END]
test:  pd.Series = close.loc[TEST_START:]

# ── Sanity checks ─────────────────────────────────────────────────────────────
assert len(train) > 0, "train is empty!"
assert len(test)  > 0, "test is empty!"
assert train.isna().sum() == 0, f"train has {train.isna().sum()} NaNs!"
assert test.isna().sum()  == 0, f"test has {test.isna().sum()} NaNs!"
assert train.index.max() < test.index.min(), "train/test overlap!"

print(f"Train: {train.index.min().date()} → {train.index.max().date()}  ({len(train):,} rows, NaNs={train.isna().sum()})")
print(f"Test : {test.index.min().date()} → {test.index.max().date()}   ({len(test):,} rows, NaNs={test.isna().sum()})")
print(f"Close dtype: {close.dtype}")
print(close.describe())


In [ ]:
# ── Fig 1: Train / Test split ─────────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(train.index, train.values, color="#457B9D", linewidth=1.2, label="Train (2015–2024)")
ax.plot(test.index,  test.values,  color="#E63946", linewidth=1.2, label="Test  (2025–2026)")
ax.axvline(pd.Timestamp(TEST_START), color="black", linestyle="--", linewidth=1, label="Split")
ax.set_title("TSLA Close Price – Train / Test Split", fontsize=14, fontweight="bold")
ax.set_xlabel("Date"); ax.set_ylabel("Price (USD)"); ax.legend()
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))
plt.tight_layout()
fig.savefig(IMG_DIR / "t2_fig1_train_test_split.png", bbox_inches="tight")
plt.show()
print("Saved t2_fig1_train_test_split.png")


## 2. ARIMA / SARIMA

In [ ]:
# ── Fig 2: ACF / PACF on first-differenced series ────────────────────────────
train_diff = train.diff().dropna()

fig, axes = plt.subplots(1, 2, figsize=(14, 4))
plot_acf(train_diff,  lags=40, ax=axes[0], title="ACF  (1st diff)")
plot_pacf(train_diff, lags=40, ax=axes[1], title="PACF (1st diff)", method="ywm")
plt.tight_layout()
fig.savefig(IMG_DIR / "t2_fig2_acf_pacf.png", bbox_inches="tight")
plt.show()
print("Saved t2_fig2_acf_pacf.png")

# ADF test
adf_stat, adf_p, *_ = adfuller(train_diff)
print(f"ADF on 1st-diff: stat={adf_stat:.4f}, p={adf_p:.4f} → {'stationary' if adf_p < 0.05 else 'non-stationary'}")


In [ ]:
# ── Select ARIMA order via auto_arima ────────────────────────────────────────
from pmdarima import auto_arima

print("Running auto_arima (may take ~1 min)...")
_selector = auto_arima(
    train,
    start_p=0, max_p=5,
    start_q=0, max_q=5,
    d=1,
    seasonal=False,
    information_criterion="aic",
    stepwise=True,
    error_action="ignore",
    suppress_warnings=True,
)
arima_order = _selector.order
print(f"Best order: {arima_order}  AIC={_selector.aic():.2f}")


In [ ]:
# ── ARIMA forecast using statsmodels (avoids pmdarima predict state issues) ───
print(f"Fitting ARIMA{arima_order} on {len(train)} training points...")
_sm_model  = SmARIMA(train, order=arima_order).fit()
_fc_result = _sm_model.get_forecast(steps=len(test))
arima_forecast = pd.Series(_fc_result.predicted_mean.values, index=test.index)

# ── Sanity checks ─────────────────────────────────────────────────────────────
assert len(arima_forecast) == len(test), "forecast length mismatch!"
assert arima_forecast.isna().sum() == 0, f"arima_forecast has {arima_forecast.isna().sum()} NaNs!"
assert (arima_forecast > 0).all(), "arima_forecast has non-positive values!"

print(f"arima_forecast: shape={arima_forecast.shape}, NaNs={arima_forecast.isna().sum()}")
print(arima_forecast.head())

# ── Fig 3: ARIMA forecast vs actuals ─────────────────────────────────────────
fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(train.iloc[-120:].index, train.iloc[-120:].values,
        color="#457B9D", linewidth=1.2, label="Train (last 120 days)")
ax.plot(test.index, test.values,
        color="#2A9D8F", linewidth=1.5, label="Actual")
ax.plot(arima_forecast.index, arima_forecast.values,
        color="#E63946", linewidth=1.5, linestyle="--",
        label=f"ARIMA{arima_order} Forecast")
ax.set_title("ARIMA Forecast vs Actual – TSLA (Test Period)", fontsize=14, fontweight="bold")
ax.set_xlabel("Date"); ax.set_ylabel("Price (USD)"); ax.legend()
plt.tight_layout()
fig.savefig(IMG_DIR / "t2_fig3_arima_forecast.png", bbox_inches="tight")
plt.show()
print("Saved t2_fig3_arima_forecast.png")


## 3. LSTM Model

In [ ]:
# ── Scale & build sliding-window sequences ────────────────────────────────────
WINDOW = 60

scaler = MinMaxScaler(feature_range=(0, 1))
train_scaled = scaler.fit_transform(train.values.reshape(-1, 1))

X_train, y_train = [], []
for i in range(WINDOW, len(train_scaled)):
    X_train.append(train_scaled[i - WINDOW:i, 0])
    y_train.append(train_scaled[i, 0])

X_train = np.array(X_train).reshape(-1, WINDOW, 1)
y_train = np.array(y_train)

print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
assert not np.isnan(X_train).any(), "X_train has NaNs!"
assert not np.isnan(y_train).any(), "y_train has NaNs!"


In [ ]:
# ── LSTM architecture ─────────────────────────────────────────────────────────
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(WINDOW, 1)),
    Dropout(0.2),
    LSTM(32, return_sequences=False),
    Dropout(0.2),
    Dense(1),
])
model.compile(optimizer="adam", loss="mse")
model.summary()


In [ ]:
# ── Train ─────────────────────────────────────────────────────────────────────
early_stop = EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stop],
    verbose=1,
)

# ── Fig 4: training loss ──────────────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(history.history["loss"],     label="Train Loss")
ax.plot(history.history["val_loss"], label="Val Loss")
ax.set_title("LSTM Training Loss", fontsize=13, fontweight="bold")
ax.set_xlabel("Epoch"); ax.set_ylabel("MSE"); ax.legend()
plt.tight_layout()
fig.savefig(IMG_DIR / "t2_fig4_lstm_loss.png", bbox_inches="tight")
plt.show()
print("Saved t2_fig4_lstm_loss.png")


In [ ]:
# ── LSTM forecast over test period ────────────────────────────────────────────
combined        = pd.concat([train, test])
combined_scaled = scaler.transform(combined.values.reshape(-1, 1))

X_test = []
start  = len(train) - WINDOW
for i in range(start, start + len(test)):
    X_test.append(combined_scaled[i:i + WINDOW, 0])

X_test = np.array(X_test).reshape(-1, WINDOW, 1)
assert X_test.shape[0] == len(test), f"X_test rows {X_test.shape[0]} != test len {len(test)}"
assert not np.isnan(X_test).any(), "X_test has NaNs!"

lstm_pred_scaled = model.predict(X_test, verbose=0)
lstm_pred_values = scaler.inverse_transform(lstm_pred_scaled).flatten()
lstm_forecast    = pd.Series(lstm_pred_values, index=test.index)

# ── Sanity checks ─────────────────────────────────────────────────────────────
assert len(lstm_forecast) == len(test), "lstm_forecast length mismatch!"
assert lstm_forecast.isna().sum() == 0, f"lstm_forecast has {lstm_forecast.isna().sum()} NaNs!"

print(f"lstm_forecast: shape={lstm_forecast.shape}, NaNs={lstm_forecast.isna().sum()}")
print(lstm_forecast.head())

# ── Fig 5: LSTM forecast vs actuals ──────────────────────────────────────────
fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(train.iloc[-120:].index, train.iloc[-120:].values,
        color="#457B9D", linewidth=1.2, label="Train (last 120 days)")
ax.plot(test.index, test.values,
        color="#2A9D8F", linewidth=1.5, label="Actual")
ax.plot(lstm_forecast.index, lstm_forecast.values,
        color="#F4A261", linewidth=1.5, linestyle="--", label="LSTM Forecast")
ax.set_title("LSTM Forecast vs Actual – TSLA (Test Period)", fontsize=14, fontweight="bold")
ax.set_xlabel("Date"); ax.set_ylabel("Price (USD)"); ax.legend()
plt.tight_layout()
fig.savefig(IMG_DIR / "t2_fig5_lstm_forecast.png", bbox_inches="tight")
plt.show()
print("Saved t2_fig5_lstm_forecast.png")


## 4. Evaluation – MAE, RMSE, MAPE

In [ ]:
# ── Pre-evaluation sanity checks ──────────────────────────────────────────────
print("=== Pre-evaluation sanity checks ===")
print(f"test           shape={test.shape},           NaNs={test.isna().sum()}")
print(f"arima_forecast shape={arima_forecast.shape}, NaNs={arima_forecast.isna().sum()}")
print(f"lstm_forecast  shape={lstm_forecast.shape},  NaNs={lstm_forecast.isna().sum()}")

for name, arr in [("test", test), ("arima_forecast", arima_forecast), ("lstm_forecast", lstm_forecast)]:
    assert arr.isna().sum() == 0, f"{name} still has NaNs — re-run its cell!"
    assert len(arr) > 0,          f"{name} is empty!"
print("All checks passed ✓")

# ── Metric helpers ────────────────────────────────────────────────────────────
def mape(actual: np.ndarray, predicted: np.ndarray) -> float:
    mask = actual != 0
    return float(np.mean(np.abs((actual[mask] - predicted[mask]) / actual[mask])) * 100)

def evaluate(name: str, actual: np.ndarray, predicted: np.ndarray) -> dict:
    a = np.array(actual,    dtype=float)
    p = np.array(predicted, dtype=float)
    assert len(a) == len(p), f"{name}: length mismatch {len(a)} vs {len(p)}"
    assert not np.isnan(a).any(), f"{name}: actual has NaNs"
    assert not np.isnan(p).any(), f"{name}: predicted has NaNs"
    mae      = float(mean_absolute_error(a, p))
    rmse     = float(np.sqrt(mean_squared_error(a, p)))
    mape_val = mape(a, p)
    print(f"  {name:<16}  MAE={mae:>8.2f}  RMSE={rmse:>8.2f}  MAPE={mape_val:>6.2f}%")
    return {"model": name, "MAE": round(mae, 4), "RMSE": round(rmse, 4), "MAPE": round(mape_val, 4)}

print(f"\n{'Model':<16}  {'MAE':>10}  {'RMSE':>10}  {'MAPE':>8}")
print("-" * 54)
arima_metrics = evaluate(f"ARIMA{arima_order}", test.values, arima_forecast.values)
lstm_metrics  = evaluate("LSTM",               test.values, lstm_forecast.values)


## 5. Model Comparison & Discussion

In [ ]:
# ── Fig 6: side-by-side forecast comparison ───────────────────────────────────
metrics_df = pd.DataFrame([arima_metrics, lstm_metrics]).set_index("model")
print("\nModel Comparison Table")
print("=" * 45)
print(metrics_df.to_string())

fig, axes = plt.subplots(1, 2, figsize=(16, 5), sharey=True)
for ax, forecast, label, color in zip(
    axes,
    [arima_forecast, lstm_forecast],
    [f"ARIMA{arima_order}", "LSTM"],
    ["#E63946", "#F4A261"],
):
    ax.plot(test.index, test.values,     color="#2A9D8F", linewidth=1.5, label="Actual")
    ax.plot(test.index, forecast.values, color=color,     linewidth=1.5,
            linestyle="--", label=label)
    ax.set_title(f"{label} vs Actual", fontsize=13, fontweight="bold")
    ax.set_xlabel("Date"); ax.set_ylabel("Price (USD)"); ax.legend()
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %Y"))
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=30)

plt.tight_layout()
fig.savefig(IMG_DIR / "t2_fig6_model_comparison.png", bbox_inches="tight")
plt.show()
print("Saved t2_fig6_model_comparison.png")


### Discussion

**ARIMA** is a classical statistical model that assumes linearity and stationarity.
After first-differencing (d=1), it captures autocorrelation structure well for short horizons.
However, it cannot model non-linear patterns or long-range dependencies in volatile assets like TSLA.

**LSTM** is a recurrent neural network that learns non-linear temporal patterns from a sliding window
of past prices. It typically outperforms ARIMA on volatile financial series over longer test horizons,
at the cost of longer training time and more hyperparameter tuning.

The model with lower RMSE on the test set is recommended for Task 3 future forecasting.

## 6. Save Outputs

In [ ]:
# ── Save forecast CSVs ────────────────────────────────────────────────────────
arima_forecast.to_csv(DATA_DIR / "arima_forecast.csv", header=["arima_forecast"])
lstm_forecast.to_csv( DATA_DIR / "lstm_forecast.csv",  header=["lstm_forecast"])
print("Saved arima_forecast.csv")
print("Saved lstm_forecast.csv")

# ── Save stats JSON ───────────────────────────────────────────────────────────
best_model = min([arima_metrics, lstm_metrics], key=lambda x: x["RMSE"])["model"]
stats = {
    "arima_order":       list(arima_order),
    "lstm_window":       WINDOW,
    "metrics":           {m["model"]: {k: v for k, v in m.items() if k != "model"}
                          for m in [arima_metrics, lstm_metrics]},
    "best_model_by_rmse": best_model,
}
with open(DATA_DIR / "task2_stats.json", "w") as f:
    json.dump(stats, f, indent=2)
print(f"Saved task2_stats.json  (best model: {best_model})")
print(json.dumps(stats, indent=2))
